1. Load mol objects and highlight sites that are being misclassified
2. A lot of the false positives are tertiary carbon atoms with high spin. Tertiary carbon atoms don't usually do as well, but the model can't tell the difference between T-carbons embedded in rings or connected to methyl groups. This could deserve its own feature
3. A lot of the false negatives are for small molecules with multiple nitrogen atoms. Correcting the aromatic ring size feature could potentially improve this. First check if the RDKit feature is trustworthy; if not, can just look for the full set of carbons with only 3 bonds, as these are necessarily sp2.
4. The aromatic ring size features appears to be completely wrong, and I found at least one counterexample for OrthoOrPara. DistanceToN may obviate OrthoOrPara anyway
5. sf189x0 has an active site at carbon 4 (binding energy -0.1eV), and a symmetric site at carbon 21 that is not active. This likely illustrates different conformers can subtely affect these energies
6. Catalyst sf41x0 lists all binding sites as inactive, but some actually are (and possibly most). If I move the correct files to the correct directory, can I just rerun Kunal's parsing scripts?

Score to beat: 93% with random forests
Since 80% of sites are not active, the threshold for success is 80%

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [2]:
from ngcc_ml import data_tools

RDKit WARNING: [16:27:27] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
df = pd.read_csv("/home/nricke/work/klodaya/notebooks_klodaya/DidItBindWithSubstructure.csv", index_col=0)

In [4]:
df

,Atom Number,Catalyst Name,CatalystO2File,Element,SpinDensity,ChElPGPositiveCharge,ChElPGNeutralCharge,ChargeDifference,Doesitbind,BondLength,...,Substructure1,Substructure2,Substructure3,Substructure4,Substructure5,Substructure6,Substructure7,Substructure8,Substructure9,Substructure10
0,1,sf100x0,None,C,-0.008245,-0.275350,-0.200227,0.075123,False,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,3,sf100x0,D:\Kunal\Documents\MIT\allfiles\O2_binding_dat...,C,0.555664,-0.064043,-0.339572,-0.275529,True,1.535452,...,0,0,0,0,0,0,0,0,0,0
2,4,sf100x0,None,C,-0.181519,0.037008,0.096249,0.059241,False,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,5,sf100x0,None,C,0.208580,-0.226453,-0.308850,-0.082397,False,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,6,sf100x0,D:\Kunal\Documents\MIT\allfiles\O2_binding_dat...,C,-0.119560,0.176015,0.163894,-0.012121,False,1.701145,...,0,0,0,0,0,0,0,0,0,0
5,7,sf100x0,D:\Kunal\Documents\MIT\allfiles\O2_binding_dat...,C,0.221689,0.308617,0.215658,-0.092959,False,1.619080,...,0,0,0,0,0,0,0,0,0,0
6,8,sf100x0,None,C,-0.138080,-0.337969,-0.357701,-0.019732,False,0.000000,...,0,0,0,0,0,0,0,0,0,0
7,9,sf100x0,None,C,0.243169,0.054121,-0.032052,-0.086173,False,0.000000,...,0,0,0,0,0,0,0,0,0,0
8,10,sf100x0,None,C,-0.012210,0.079364,0.079428,0.000064,False,0.000000,...,0,0,0,0,0,0,0,0,0,0
9,11,sf100x0,D:\Kunal\Documents\MIT\allfiles\O2_binding_dat...,C,-0.006535,0.093478,0.074304,-0.019174,False,1.633351,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.columns

Index(['Atom Number', 'Catalyst Name', 'CatalystO2File', 'Element',
       'SpinDensity', 'ChElPGPositiveCharge', 'ChElPGNeutralCharge',
       'ChargeDifference', 'Doesitbind', 'BondLength', 'IonizedFreeEnergy',
       'IonizationEnergy', 'BindingEnergy', 'NeutralFreeEnergy', 'OrthoOrPara',
       'Meta', 'FartherThanPara', 'DistanceToN', 'AverageBondLength',
       'BondLengthRange', 'NumberOfHydrogens', 'AromaticSize', 'IsInRingSize6',
       'IsInRingSize5', 'NeighborSpinDensity', 'NeighborChElPGCharge',
       'NeighborChargeDifference', 'Substructure1', 'Substructure2',
       'Substructure3', 'Substructure4', 'Substructure5', 'Substructure6',
       'Substructure7', 'Substructure8', 'Substructure9', 'Substructure10'],
      dtype='object')

In [6]:
df.Doesitbind.value_counts()

False    3455
True      697
Name: Doesitbind, dtype: int64

In [9]:
df[df["Catalyst Name"] == "sf41x0"][["Atom Number", "Doesitbind"]]

,Atom Number,Doesitbind
3271,2,False
3272,3,False
3273,4,False
3274,5,False
3275,6,False
3276,7,False
3277,8,False
3278,9,False
3279,11,False


In [ ]:
feature_cols = ["SpinDensity", "ChElPGNeutralCharge", "ChargeDifference", "IonizationEnergy", "OrthoOrPara", "Meta", "FartherThanPara", "DistanceToN", "NumberOfHydrogens", "AromaticSize", "IsInRingSize6", "IsInRingSize5", "NeighborSpinDensity", 'NeighborChElPGCharge', 'NeighborChargeDifference', "AromaticSize"]
X = df[feature_cols]
y = df["Doesitbind"].astype('int')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=100, class_weight={0:0.5, 1:0.5})
rfc.fit(X_train, y_train)
print('Accuracy of RFC on test set: {:.2f}'.format(rfc.score(X_test, y_test)))
print('Accuracy of RFC on training set: {:.2f}'.format(rfc.score(X_train, y_train)))
scores = cross_val_score(rfc, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
# Get indices of misclassified active sites
X_test_y = X_test.copy()
X_test_y["y_pred"] = y_pred
X_test_y["y_test"] = y_test

In [ ]:
X_test_y

In [ ]:
dfy = df.merge(X_test_y[["y_pred", "y_test"]], how="inner", left_index=True, right_index=True)

In [ ]:
dfy_miss = dfy[dfy["y_pred"] != dfy["y_test"]]
dfy_false_pos = dfy_miss[dfy_miss["y_pred"] == 1]
dfy_false_neg = dfy_miss[dfy_miss["y_pred"] == 0]

In [ ]:
dfy_miss

In [ ]:
dfy_miss.loc[70]

In [ ]:
686.93232484 - 686.81836881

In [ ]:
dfy_false_neg[["Atom Number", "AromaticSize", "Catalyst Name", "BindingEnergy", "IonizationEnergy", "SpinDensity"]].sort_values(by="Catalyst Name")

In [ ]:
dfy_false_pos[["Atom Number", "AromaticSize", "Catalyst Name", "BindingEnergy", "IonizationEnergy", "SpinDensity", "Meta", "OrthoOrPara", "DistanceToN", "NumberOfHydrogens"]].sort_values(by="Catalyst Name")

In [ ]:
df[df["Catalyst Name"] == "sf41x0"][["Atom Number", "Doesitbind", "BindingEnergy"]]